**Please upvote my notebook if You like it!**

In [ ]:
!pip3 install -U lightautoml

# QUICK WORKAROUND FOR PROBLEM WITH PANDAS
!pip3 install -U pandas

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Downloading data**
+ adding data from "stroke prediction dataset"

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e2/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e2/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e2/sample_submission.csv')
addition_data = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
addition_data.isna().any()

In [ ]:
addition_data.bmi = addition_data.bmi.fillna(np.mean(addition_data.bmi))

In [ ]:
train_df['generated'] = 1
test_df['generated'] = 1
addition_data['generated'] = 0
train_df = pd.concat([train_df, addition_data],axis=0, ignore_index=True)

In [ ]:
df = pd.concat([train_df, test_df], axis=0)
df

In [ ]:
df = pd.get_dummies(df)
df = df.drop('id', axis=1)

In [ ]:
df

In [ ]:
train_df = df.iloc[:-len(test_df),:]
test_df = df.iloc[-len(test_df):,:].drop('stroke', axis=1).reset_index(drop=True)

X = train_df.drop('stroke', axis=1)
y = train_df.stroke
X_test = test_df

**The model**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
task = Task('binary', metric = 'auc')

N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 60 * 120
TARGET_NAME = 'stroke'

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)
# TabularUtilizedAutoML
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
#     reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
#     general_params = {"use_algos": [all_models]},
    reader_params = {'n_jobs': N_THREADS}
)


In [ ]:
roles = {
    'target': TARGET_NAME
}

In [ ]:
preds_tr = automl.fit_predict(train_df, roles = roles, verbose = 1)

In [ ]:
preds_tr

In [ ]:
preds = automl.predict(X_test)
preds

In [ ]:
submission['stroke'] = preds.data[:,0]
submission

In [ ]:
submission.to_csv('submission.csv', index=False)